Converting the images in augmented into a dataframe so that the model can train

In [11]:
from PIL import Image
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
print(tf.test.is_built_with_cuda())
from keras_preprocessing.image import img_to_array

def images_to_dataframe(image_path, image_dataset, image_size):
    label_set = np.empty((0,), dtype = int)

    for image in os.listdir(image_path):

        label = ""
        label_list = ["bowser", "luigi", "mario", "pikachu", "sonic"]

        for name in label_list:
            if name in image:
                label = np.array([name])

        label_set = np.append(label_set, label)

        image = Image.open(image_path + image)
        image = image.resize((image_size, image_size))
        image_arr = img_to_array(image)
        image_arr = image_arr.reshape(1, image_size, image_size, 3)
        image_dataset = np.append(image_dataset, image_arr, axis = 0)
        
    return image_dataset, label_set

    
image_path = "C:/Users/joshu/Python Projects/Machine Learning Projects/Web Image Classifier/augmented/"
image_size = 128
image_dataset = np.empty((0, image_size, image_size, 3), dtype = int)
image_dataset, label_set = images_to_dataframe(image_path, image_dataset, image_size)

print(image_dataset.shape)
print(label_set.shape)


True
(2053, 128, 128, 3)
(2053,)


Changing the labels so that cross entropy loss works

In [12]:
label_list = ["bowser", "luigi", "mario", "pikachu", "sonic"]
for i in range(len(label_set)):
    for j in range(5):
        if label_set[i] == label_list[j]:
            label_set[i] = j

print(label_set)
print(label_set.shape)


['0' '0' '0' ... '4' '4' '4']
(2053,)


Importing dependencies

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import numpy as np
import matplotlib.pyplot as plt

Performing Crossentropy loss

In [14]:
def show_minmax(arr,num):
    image = arr[num]
    print(image.min(), image.max())

numClasses = 5
input_shape = (image_size, image_size, 3)
show_minmax(image_dataset, 1)
image_dataset = image_dataset.astype("float32")
image_dataset /= 255
show_minmax(image_dataset, 1)

label_set = keras.utils.to_categorical(label_set, numClasses)
print(label_set.shape)

0.0 255.0
0.0 1.0
(2053, 5)


Splitting the data into training and testing data

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(image_dataset, label_set, test_size=0.05, random_state=4)
print(X_train.shape)
print(Y_train.shape)

(1950, 128, 128, 3)
(1950, 5)


Importing dependencies

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, SpatialDropout2D, AlphaDropout

Training the model

In [17]:
epochs = 30
model = Sequential()
model.add(Conv2D(input_shape = input_shape, filters = 32, kernel_size = (2,2), activation = "relu"))
model.add(Conv2D(filters = 32, kernel_size = (2,2), activation = "relu"))
model.add(MaxPooling2D(pool_size = (3,3)))
model.add(Dropout(rate = 0.3))
model.add(Conv2D(filters = 32, kernel_size = (2,2), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(rate = 0.3))
model.add(Flatten())
model.add(Dense(16, activation = "relu"))
model.add(Dense(5, activation = "softmax"))
model.summary()

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = (["accuracy"]))
model.fit(image_dataset, label_set, epochs = epochs, shuffle = True)
model.save("Img_Classification.h5")

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 127, 127, 32)      416       
                                                                 
 conv2d_7 (Conv2D)           (None, 126, 126, 32)      4128      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 42, 42, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 42, 42, 32)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 41, 41, 32)        4128      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 20, 20, 32)       0         
 2D)                                                  